Early-Stopping Function

Importing Necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import random
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
df = pd.read_csv('outcclean.csv')

Splitting Data->Train,Validate,Test

In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'].values,
    df['emotion'].values,
    test_size=0.2,
    random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.5,
    random_state=42
)


In [ ]:
print(f"Number of training samples: {len(train_texts)}")
print(f"Number of validation samples: {len(val_texts)}")
print(f"Number of test samples: {len(test_texts)}")

Label Encoding For Emotions

In [ ]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)


In [ ]:
df['emotion'] = label_encoder.fit_transform(df['emotion'])

In [ ]:
# Get the mapping of encoded labels to original emotions
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Label Encoding Mapping:")
for emotion, label in label_mapping.items():
    print(f"{emotion}: {label}")


In [ ]:
train_labels


Loading and Initialization of Indicbert Model and Tokenizer

In [ ]:
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name, keep_accents=True)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)
model.config.output_hidden_states = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

In [ ]:
print(tokenizer)

In [ ]:
print(model.config)

Defining a Custom Dataset Class And Generating Sentence Embedding

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, model, device):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.model = model
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):



        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize and encode text
        encoding = self.tokenizer(text, return_tensors='pt', truncation=True, padding='max_length',  max_length=128)
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        # Move tensors to device
        input_ids = input_ids.to(self.device)
        attention_mask = attention_mask.to(self.device)

        # Forward pass through model to get sentence embeddings
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
        hidden_states = outputs.hidden_states
        second_to_last_layer = hidden_states[-2]
        sentence_embedding = second_to_last_layer.squeeze()

        #Return Processed Data
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long, device=self.device),
            'embedding': sentence_embedding
        }


Creating Data loaders

In [ ]:
#creating data instances
train_dataset = EmotionDataset(train_texts, train_labels, tokenizer, model, device)
val_dataset = EmotionDataset(val_texts, val_labels, tokenizer, model, device)
test_dataset = EmotionDataset(test_texts, test_labels, tokenizer, model, device)

batch_size = 18
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_loader

Defining the Model Architecture(structure and forward pass of neural network)

In [ ]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob=0.5):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.ln1 = nn.LayerNorm(hidden_dim1)
        self.relu1 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(dropout_prob)

        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.ln2 = nn.LayerNorm(hidden_dim2)
        self.relu2 = nn.ELU()
        self.dropout2 = nn.Dropout(dropout_prob)

        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.ln1(out)
        out = self.relu1(out)
        out = self.dropout1(out)

        out = self.fc2(out)
        out = self.ln2(out)
        out = self.relu2(out)
        out = self.dropout2(out)

        out = self.fc3(out)
        return out

Setting the dimensions and hyperparameter

In [ ]:
input_dim = model.config.hidden_size
hidden_dim1 = 256
hidden_dim2 = 128
output_dim = len(label_encoder.classes_)
dropout_prob = 0.1

In [ ]:
mlp_model = MLPClassifier(input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob).to(device)

Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)



Training Epochs

In [ ]:
num_epochs = 18
best_val_accuracy = 0.0

Training Iteration

In [ ]:
import torch.nn.functional as F
for epoch in range(num_epochs):
    mlp_model.train()
    train_loss = 0.0

    for batch in train_loader:
        embeddings = batch['embedding'].to(device)
        labels = batch['labels'].to(device)

        print(f"Original embeddings shape: {embeddings.shape}")
        embeddings = torch.mean(embeddings, dim=1)
#Foeward pass
        optimizer.zero_grad()
        outputs = mlp_model(embeddings)

        print(f"Outputs shape: {outputs.shape}")
        print(f"Labels shape: {labels.shape}")
#Loss calculation and backpropagation
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

#calculating average training loss
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_train_loss}")



Evaluation of Model

In [ ]:
mlp_model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        embeddings = batch['embedding'].to(device)  # Shape: [batch_size, 128, 768]
        labels = batch['labels'].to(device)  # Shape: [batch_size]

        # Average over the sequence length dimension (128)
        embeddings = torch.mean(embeddings, dim=1)  # Shape: [batch_size, 768]

        outputs = mlp_model(embeddings)  # Shape: [batch_size, 4]

        loss = criterion(outputs, labels)
        val_loss += loss.item()

# Calculating accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_val_loss = val_loss / len(val_loader)
accuracy = 100 * correct / total
print(f"Validation Loss: {avg_val_loss}, Accuracy: {accuracy}%")





Evaluation of model->Accuracy & F1-Score of data

In [ ]:
import torch
from torch import nn
from sklearn.metrics import precision_recall_fscore_support

def evaluate_model(model, criterion, data_loader, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in data_loader:
            embeddings = batch['embedding'].to(device)  # Shape: [batch_size, 128, 768]
            labels = batch['labels'].to(device)  # Shape: [batch_size]

            # Average over the sequence length dimension (128)
            embeddings = torch.mean(embeddings, dim=1)  # Shape: [batch_size, 768]

            outputs = model(embeddings)  # Shape: [batch_size, 4]

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Collect all labels and predictions for F1 score calculation
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

    return avg_loss, accuracy, f1

# Define your criterion (loss function) and device (CPU or GPU)
criterion = nn.CrossEntropyLoss()  # Assuming you're using CrossEntropyLoss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Evaluate on validation dataset
val_loss, val_accuracy, val_f1 = evaluate_model(mlp_model, criterion, val_loader, device)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}%, Validation F1 Score: {val_f1}")

# Evaluate on test dataset
test_loss, test_accuracy, test_f1 = evaluate_model(mlp_model, criterion, test_loader, device)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}%, Test F1 Score: {test_f1}")

# Evaluate on train dataset
train_loss, train_accuracy, train_f1 = evaluate_model(mlp_model, criterion, train_loader, device)
print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}%, Train F1 Score: {train_f1}")


Evaluation of model performance per Class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

all_preds = []
all_labels = []

mlp_model.eval()
with torch.no_grad():
    for batch in val_loader:
        embeddings = batch['embedding'].to(device)
        labels = batch['labels'].to(device)

        embeddings = torch.mean(embeddings, dim=1)
        outputs = mlp_model(embeddings)

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3'], yticklabels=['Class 0', 'Class 1', 'Class 2', 'Class 3'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Assuming all_labels and all_preds are defined
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']

# Generate the classification report
report = classification_report(all_labels, all_preds, target_names=target_names)
print(report)

# Compute the confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

# Calculate accuracy for each class
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# Print accuracy for each class
for i, accuracy in enumerate(class_accuracies):
    print(f"Accuracy of {target_names[i]}: {accuracy:.2f}")
